# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [42]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [43]:
geoapify_key

'92220b0a932e41808964d61ad36beb6c'

In [36]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,25.59,71,100,1.89,PN,1707011594
1,1,talnakh,69.4865,88.3972,-14.33,96,100,3.88,RU,1707011514
2,2,grytviken,-54.2811,-36.5092,10.16,75,94,3.88,GS,1707011594
3,3,antsiranana,-12.2787,49.2917,24.71,89,100,0.46,MG,1707011620
4,4,devrek,41.2192,31.9558,2.62,83,100,2.37,TR,1707012187


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
city_data_df['Lat'] = city_data_df['Lat'].astype(float)
city_data_df['Lng'] = city_data_df['Lng'].astype(float)

plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity', 
                                  hover_cols=['City', 'Humidity'], 
                                  color='blue', alpha=0.5, tiles='OSM', 
                                  frame_width=700, frame_height=500)

plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
ideal_temp_min = 15
ideal_temp_max = 25
ideal_humidity_max = 40
ideal_cloudiness_max = 75
ideal_wind_speed_max = 5

ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= ideal_temp_min) &
    (city_data_df["Max Temp"] <= ideal_temp_max) &
    (city_data_df["Humidity"] <= ideal_humidity_max) &
    (city_data_df["Cloudiness"] <= ideal_cloudiness_max) &
    (city_data_df["Wind Speed"] <= ideal_wind_speed_max)
].copy()

ideal_weather_df.dropna(inplace=True)

ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
83,83,rakops,-21.0167,24.3333,23.46,39,18,3.47,BW,1707012199
86,86,damasak,13.0977,12.4952,16.94,27,1,4.35,NG,1707012199
145,145,tura,25.5198,90.2201,15.18,33,0,0.88,IN,1707011661
226,226,ke-macina,13.9623,-5.3572,21.62,13,5,4.67,ML,1707012217
246,246,ingeniero jacobacci,-41.3427,-69.5448,19.45,14,24,2.60,AR,1707011634


Step 3: Create a new DataFrame called `hotel_df`

In [39]:
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

hotel_df['Hotel Name'] = ""

hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
83,rakops,BW,-21.0167,24.3333,39,
86,damasak,NG,13.0977,12.4952,27,
145,tura,IN,25.5198,90.2201,33,
226,ke-macina,ML,13.9623,-5.3572,13,
246,ingeniero jacobacci,AR,-41.3427,-69.5448,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
print("Starting hotel search")

radius = 10000

for index, row in hotel_df.iterrows():
    lat, lng = row['Lat'], row['Lng']
    
    params = {
        'categories': 'accommodation.hotel',
        'filter': f'circle:{lng},{lat},{radius}',
        'limit': 1,  
        'apiKey': geoapify_key
    }
    
    base_url = "https://api.geoapify.com/v2/places"
    try:
        response = requests.get(base_url, params=params)
        data = response.json() 
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (requests.RequestException, IndexError, KeyError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
print("---------------\n---------------")
print(hotel_df.head())

Starting hotel search
rakops - nearest hotel: Rakops River Lodge
damasak - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
ke-macina - nearest hotel: No hotel found
ingeniero jacobacci - nearest hotel: Cumelcan
bignona - nearest hotel: No hotel found
tan-tan - nearest hotel: فندق
kyabe - nearest hotel: No hotel found
thames - nearest hotel: Kopu Station Hotel
anchau - nearest hotel: No hotel found
taungoo - nearest hotel: Pyi Tawwin Hotel
zalanga - nearest hotel: No hotel found
guider - nearest hotel: No hotel found
kerikeri - nearest hotel: Avalon Resort
keffi - nearest hotel: No hotel found
---------------
---------------
                    City Country      Lat      Lng  Humidity  \
83                rakops      BW -21.0167  24.3333        39   
86               damasak      NG  13.0977  12.4952        27   
145                 tura      IN  25.5198  90.2201        33   
226            ke-macina      ML  13.9623  -5.3572        13   
246  ingeniero jacobacci     

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
map_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, 
                                  size=100,
                                  color='blue', alpha=0.5, 
                                  tiles='OSM', 
                                  hover_cols=['City', 'Country', 'Hotel Name'], 
                                  frame_width=700, frame_height=500)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)